In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This script is used to  get samples which are later used to estimate DIC of SCRaPL without inflation on mEBC data. The only difference from standard inference scripts is that it stores some extra parameters like log likelihood tied to a particular sample and average of posterior latent accessibility and expression.

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import scipy
import scipy.stats

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from tensorflow import keras

from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import pickle
from timeit import default_timer as timer

In [ ]:
tfd = tfp.distributions
tfb = tfp.bijectors
Folder = '/content/drive/MyDrive/'

In [ ]:
#Load accessibility, expression data and cell normalization constants
yy_acc_pd = pd.read_csv(Folder+'SCRaPL/Real/Data/Acc_atac.csv',',',header=[0],index_col=[0])
yy_exp_pd = pd.read_csv(Folder+'SCRaPL/Real/Data/Rna_atac.csv',',',header=[0],index_col=[0])
Norm = pd.read_csv(Folder+'SCRaPL/Real/Data/Nrm_atac.csv',',',index_col=[1])
Norm = Norm.drop(columns=Norm.columns[0])

In [ ]:
yy_acc = tf.convert_to_tensor(yy_acc_pd,dtype=tf.float32)
yy_exp = tf.convert_to_tensor(yy_exp_pd,dtype=tf.float32)
Norm_acc = tf.convert_to_tensor(Norm.iloc[:,0],dtype=tf.float32)
Norm_exp = tf.convert_to_tensor(Norm.iloc[:,1],dtype=tf.float32)

In [ ]:
x_genes,x_cells = tf.shape(yy_acc)

In [ ]:
aff = tfb.Chain([tfb.Shift(-1.),tfb.Scale(scale=2.)])
aff_inv = tfb.Invert(aff)

exp = tfb.Exp()
log = tfb.Invert(exp)

tanh = tfb.Tanh()
tanh_inv = tfb.Invert(tanh)

sigm = tfb.Sigmoid()
sigm_inv = tfb.Invert(sigm)

cor_trsf = tfb.Chain([aff_inv,tanh,tfb.Scale(scale=0.5)])
cor_trsf_inv = tfb.Invert(cor_trsf)

eps=0.001
bin_bij = tfb.Chain([tfb.Shift(eps/2.0),tfb.Scale(scale=1.0-eps),tfb.NormalCDF()])

cor_bij = tfb.Chain([tanh,tfb.Scale(scale=0.5)])
std_bij = tfb.Chain([exp,tfb.Scale(scale=-1.0)])
sqr_bij = tfb.Square()


In [ ]:
#SCRaPL's graphical model
def SCRaPL(N_genes,N_cells,Nrm_acc,Nrm_rna):
    def prior():
        cor_lt = yield tfd.TransformedDistribution( distribution = tfd.Beta( concentration0 = 15.0*tf.ones([N_genes,1]), concentration1=15.0*tf.ones([N_genes,1])), bijector= cor_trsf_inv, name = "cor_lt" )
        m_acc_lt = yield tfd.Normal(loc=3*tf.ones([N_genes,1]),scale=tf.ones([N_genes,1]), name = "m_acc_lt")
        m_exp_lt = yield tfd.Normal(loc=4*tf.ones([N_genes,1]),scale=tf.ones([N_genes,1]), name = "m_exp_lt")
        s_acc_lt = yield tfd.TransformedDistribution( distribution = tfd.InverseGamma(concentration=2.5*tf.ones([N_genes,1]),scale=4.5*tf.ones([N_genes,1])),bijector= log, name = "s_acc_lt" )
        s_exp_lt = yield tfd.TransformedDistribution( distribution = tfd.InverseGamma(concentration=2.5*tf.ones([N_genes,1]),scale=4.5*tf.ones([N_genes,1])),bijector= log , name = "s_exp_lt")

        cor = cor_bij.forward(cor_lt)
        s_acc = std_bij.forward(s_acc_lt)
        s_exp = std_bij.forward(s_exp_lt)
        
        mm_acc = tf.math.multiply( m_acc_lt,tf.ones([N_genes,N_cells]))
        mm_exp = tf.math.multiply( m_exp_lt,tf.ones([N_genes,N_cells]))
        ss_acc = tf.math.multiply( s_acc   ,tf.ones([N_genes,N_cells]))
        ss_exp = tf.math.multiply( s_exp   ,tf.ones([N_genes,N_cells]))
        ccor =   tf.math.multiply( cor     ,tf.ones([N_genes,N_cells]))  

        nrm_acc = tf.math.multiply( log.forward(Nrm_acc),tf.ones([N_genes,1]))
        nrm_rna = tf.math.multiply( log.forward(Nrm_rna),tf.ones([N_genes,1]))

        x_acc = yield tfd.Normal(loc = mm_acc, scale = ss_acc,name="x_acc")
        m_cnd_exp = mm_exp+tf.math.multiply(tf.math.divide(tf.math.multiply(ss_exp,x_acc-mm_acc),ss_acc),ccor)
        s_cnd_exp = tf.math.sqrt(tf.math.multiply(1-tf.math.square(ccor),tf.math.square(ss_exp)))

        x_exp = yield tfd.Normal(loc = m_cnd_exp, scale = s_cnd_exp,name="x_exp")

        y_acc = yield tfd.Poisson(log_rate=x_acc+nrm_acc, name="y_acc")
        y_exp = yield tfd.Poisson(log_rate=x_exp+nrm_rna, name="y_exp")

    comp_var_coroutine = tfd.JointDistributionCoroutineAutoBatched(prior)
    return comp_var_coroutine


In [ ]:
#SCRaPL without priors used in DIC estimation.
def SCRaPL_dic(N_genes,N_cells,Nrm_acc,Nrm_rna,param):

    cor_lt,m_acc_lt,m_exp_lt,s_acc_lt,s_exp_lt = param

    def prior():

        Root = tfd.JointDistributionCoroutine.Root
        cor = cor_bij.forward(cor_lt)
        s_acc = std_bij.forward(s_acc_lt)
        s_exp = std_bij.forward(s_exp_lt)
        
        mm_acc = tf.math.multiply( m_acc_lt,tf.ones([N_genes,N_cells]))
        mm_exp = tf.math.multiply( m_exp_lt,tf.ones([N_genes,N_cells]))
        ss_acc = tf.math.multiply( s_acc   ,tf.ones([N_genes,N_cells]))
        ss_exp = tf.math.multiply( s_exp   ,tf.ones([N_genes,N_cells]))
        ccor =   tf.math.multiply( cor     ,tf.ones([N_genes,N_cells]))

        nrm_acc = tf.math.multiply( log.forward(Nrm_acc),tf.ones([N_genes,1]))
        nrm_rna = tf.math.multiply( log.forward(Nrm_rna),tf.ones([N_genes,1]))

        x_acc = yield Root(tfd.Independent(tfd.Normal(loc = mm_acc, scale = ss_acc,name="x_acc") ,reinterpreted_batch_ndims=1))
        m_cnd_exp = mm_exp+tf.math.multiply(tf.math.divide(tf.math.multiply(ss_exp,x_acc-mm_acc),ss_acc),ccor)
        s_cnd_exp = tf.math.sqrt(tf.math.multiply(1-tf.math.square(ccor),tf.math.square(ss_exp)))

        x_exp = yield Root(tfd.Independent(tfd.Normal(loc = m_cnd_exp, scale = s_cnd_exp,name="x_exp"),reinterpreted_batch_ndims=1))


        y_acc = yield Root(tfd.Independent( tfd.Poisson(log_rate=x_acc+nrm_acc, name="y_acc") ,reinterpreted_batch_ndims=1))
        y_exp = yield Root(tfd.Independent( tfd.Poisson(log_rate=x_exp+nrm_rna, name="y_exp") ,reinterpreted_batch_ndims=1))

    comp_var_coroutine = tfd.JointDistributionCoroutine(prior)
    return comp_var_coroutine


In [ ]:
#As the dataset is too large to fit in memory we utilize genomic region independence assumption to perform sample chunks of 10 in each turn.
#As this step tends to be computationally demanding we halve the number of genomic regions to avoid the risk of running out of memory. Depending computational resources
#chunk size (www) has to be tuned 

prt = 1
time = []
skp_ind = []
www = 10
prts_tot = tf.math.ceil(x_genes/www)
jj = 0
while prt < prts_tot+1:
          aa =  (prt-1)*www
          aa1 = prt*www
          yy_acc_prt = yy_acc[aa:aa1,:]
          yy_exp_prt = yy_exp[aa:aa1,:]
          batch_ft = tf.shape(yy_exp_prt)[0]

          mdl_tr = SCRaPL(batch_ft,x_cells,Norm_acc,Norm_exp)
          init = mdl_tr.sample()
          vrr = [init[0],init[1],init[2],init[3],init[4],init[5],init[6]]

          unconstrained_bijectors = [tfb.Identity(),tfb.Identity(),tfb.Identity(),tfb.Identity(),tfb.Identity(),tfb.Identity(),tfb.Identity()]
          @tf.function(autograph=False, jit_compile=True) 

          def sample_nuts():

                  init_x = vrr
                  num_burnin_iter = 3000
                  num_warmup_iter = int(0.8*num_burnin_iter)
                  num_chain_iter = 2000

                  target_accept_rate = 0.65 

                  log_post = lambda x0,x1,x2,x3,x4,x5,x6: mdl_tr.log_prob(x0,x1,x2,x3,x4,x5,x6,yy_acc_prt,yy_exp_prt)

                  def trace_fn(_, pkr):
                        return (
                            pkr.inner_results.inner_results.target_log_prob,
                            pkr.inner_results.inner_results.leapfrogs_taken,
                            pkr.inner_results.inner_results.has_divergence,
                            pkr.inner_results.inner_results.energy,
                            pkr.inner_results.inner_results.log_accept_ratio,
                            pkr.inner_results.inner_results.step_size
                              )
                  nuts= tfp.mcmc.NoUTurnSampler(
                                            target_log_prob_fn=log_post,
                                            step_size=0.05,
                                            max_tree_depth=6
                                                ) 
                  ttk = tfp.mcmc.TransformedTransitionKernel(
                                            inner_kernel=nuts,
                                            bijector=unconstrained_bijectors
                                                                )
                  adapted_kernel=tfp.mcmc.DualAveragingStepSizeAdaptation(
                                            inner_kernel=ttk,
                                            num_adaptation_steps=num_warmup_iter,
                                            target_accept_prob= target_accept_rate)
                  
                  states , sampler_stat =tfp.mcmc.sample_chain(
                                    num_results=num_chain_iter,
                                    num_burnin_steps=num_burnin_iter,
                                    current_state=init_x,
                                    kernel=adapted_kernel,
                                    trace_fn=trace_fn) 

                  return states, sampler_stat

          start = timer()
          samples, sampler_stat = sample_nuts() 
          end = timer()
          ttime = end-start

          p_accept = tf.math.exp(tfp.math.reduce_logmeanexp(tf.minimum(sampler_stat[4], 0.)))
          stp_sz = sampler_stat[5][0]
          hmc_cor,hmc_m_acc,hmc_m_exp,hmc_s_acc,hmc_s_exp,hmc_x_acc,hmc_x_exp = samples

          crr_nuts = tf.squeeze(hmc_cor)
          s_exp_nuts = tf.squeeze(hmc_s_exp)
          s_acc_nuts = tf.squeeze(hmc_s_acc)
          m_acc_nuts = tf.squeeze(hmc_m_acc)
          m_exp_nuts = tf.squeeze(hmc_m_exp)

          mdl_dic = SCRaPL_dic(batch_ft,x_cells,Norm_acc,Norm_exp,[hmc_cor,hmc_m_acc,hmc_m_exp,hmc_s_acc,hmc_s_exp])
          llk_atac = mdl_dic.log_prob(hmc_x_acc,hmc_x_exp,yy_acc_prt,yy_exp_prt)

          x_acc_mn = tf.reduce_mean(hmc_x_acc,axis=0)
          x_exp_mn = tf.reduce_mean(hmc_x_exp,axis=0)

          qc_acc_rt = tf.math.logical_and(p_accept<0.9,p_accept>0.4)
          qc_stp_sz = stp_sz>0.00001

          if tf.math.logical_and(qc_acc_rt,qc_stp_sz)== True:
                time.append(ttime)
                with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_cor_atac_ninf_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(crr_nuts, handle)
                with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_m_acc_atac_ninf_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(m_acc_nuts, handle)
                with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_m_exp_atac_ninf_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(m_exp_nuts, handle)

                with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_s_acc_atac_ninf_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(s_acc_nuts, handle)
                with open(Folder+'SCRaPL/Real/Results_atac_DIC/nuts_s_exp_atac_ninf_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(s_exp_nuts, handle)

                with open(Folder+'SCRaPL/Real/Results_atac_DIC/avg_exp_ninf_atac_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(x_exp_mn, handle)
                with open(Folder+'SCRaPL/Real/Results_atac_DIC/avg_acc_ninf_atac_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(x_acc_mn, handle)   

                with open(Folder+'SCRaPL/Real/Results_atac_DIC/log_prob_atac_ninf_'+str(prt)+'.pickle', 'wb') as handle:
                    pickle.dump(llk_atac, handle)

                avg_time = tf.reduce_mean(tf.stack(time))
                rem_time = (tf.cast(prts_tot,dtype=tf.float32)-tf.cast(prt,dtype=tf.float32)) *avg_time
                print(tf.squeeze([rem_time,prt,stp_sz,p_accept]),tf.math.logical_and(qc_acc_rt,qc_stp_sz))
                prt+=1
                jj = 0.0
          else:
                time = []
                time.append(ttime)
                
                if jj>3.0:
                    skp_ind.append(prt)
                    prt+=1
                    jj=0.0

                avg_time = tf.reduce_mean(tf.stack(time))
                rem_time = (tf.cast(prts_tot,dtype=tf.float32)-tf.cast(prt,dtype=tf.float32)) *avg_time
                print(tf.squeeze([rem_time,prt,stp_sz,p_accept]),tf.math.logical_and(qc_acc_rt,qc_stp_sz))
                jj+=1


tf.Tensor([6.2519663e+03 3.8800000e+02 8.1360303e-03 8.1099325e-01], shape=(4,), dtype=float32) tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor([6.1407144e+03 3.8900000e+02 1.0771394e-02 6.5719372e-01], shape=(4,), dtype=float32) tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor([6.000015e+03 3.900000e+02 9.778487e-03 7.113549e-01], shape=(4,), dtype=float32) tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor([5.8326694e+03 3.9100000e+02 1.0387957e-02 6.8048680e-01], shape=(4,), dtype=float32) tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor([5.6547524e+03 3.9200000e+02 7.4985260e-03 8.3640188e-01], shape=(4,), dtype=float32) tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor([5.4886753e+03 3.9300000e+02 1.0951280e-02 6.6067606e-01], shape=(4,), dtype=float32) tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor([5.3181768e+03 3.9400000e+02 9.9221645e-03 7.1063042e-01], shape=(4,), dtype=float32) tf.Tensor(True, shape=(), dtype=bool)
tf.Tensor([5.1467642e+03 3.9500000e+02 1.0573745e-02 6.6906726e-01